# 9. PyTorch로 구현하는 합성곱 신경망(Convolutional Neural Network)

## 1. 합성곱 신경망에 대해

### 1.1 순방향 신경망(FNN)에서의 변이

#### 1층 심층 신경망

<img src='./images/09-01.png'>

#### CNN 기본

- **FNN**이전에 추가적인 **합성곱(convolution)**과 **Pooling(풀링)** 계층
- **선형 함수와 비선형** 계층: **Fully connected layer**

<img src='./images/09-02.png'>

### 1.2 1층 합성곱 계층: High Level 관점에서

- 입력 깊이 = 1 <==> 단색 이미지

<img src = "./images/09-03.png">

<img src = "./images/09-04.png">

<img src = "./images/09-05.png">

<img src = "./images/09-06.png">

<img src = "./images/09-07.png">

<img src = "./images/09-03.png">

이런식의 커널이 다양해지면 여러개의 Feature Maps(Activation Maps)가 쌓인다

- 입력 깊이 = 3 <==> RGB, 흑백이 아님

<img src = "./images/09-08.png">

- Using the same filter

<img src = "./images/09-09.png">

<img src = "./images/09-10.png">

<img src = "./images/09-11.png">

<img src = "./images/09-12.png" width=10%>

- 3층이 되어도 한장의 Feature Map에 올라간다

<img src = "./images/09-08.png">

### 1.2 1층 합성곱 계층: High Level 관점

<img src = "./images/09-03.png">

- **커널이 슬라이딩/회전(convolving)**을 이미지에 관해 수행하면서 $\rightarrow$ 2 가지 연산이 **한 패치** 마다 수행된다.
    1. 요소별 곱셈
    2. 합
- 더 많은 **커널들** = 더 많은 **피처 맵 채널들**
    - 입력값에 대한 **더 많은 정보** 를 얻을 수 있다

### 1.3 다층 합성곱 계층: High Level 관점에서

- 깊이 $\uparrow$ , 커널들의 수 $\uparrow$ => 더 많은 정보들

<img src = "./images/09-13.png">

### 1.4 Pooling 계층: High Level 관점에서

- 2가지 흔한 유형
    - 최대값 Pooling
    - 평균 Pooling

<img src = "./images/09-14.png">

<img src = "./images/09-15.png">

<img src = "./images/09-16.png">

<img src = "./images/09-17.png">

<img src = "./images/09-18.png">

<img src = "./images/09-19.png">

<img src = "./images/09-20.png">

<img src = "./images/09-21.png">

<img src = "./images/09-22.png">

### 1.5 다계층 풀링 레이어: High Level 관점에서

<img src = "./images/09-13.png">

### 1.6 패딩

<img src = "./images/09-23.png">

<img src = "./images/09-24.png">

<img src = "./images/09-25.png">

<img src = "./images/09-26.png">

<img src = "./images/09-27.png">

<img src = "./images/09-28.png">

<img src = "./images/09-29.png">

<img src = "./images/09-30.png">

<img src = "./images/09-31.png">

<img src = "./images/09-32.png">

<img src = "./images/09-33.png">

<img src = "./images/09-34.png">

### 1.7 패딩 요약

- **Valid** 패딩 (제로 패딩)
    - Output size < Input Size
- **동일** 패딩
    - Output size = Input Size

### 1.8 차원수 계산

- $O = \frac{W-K+2P}{S} + 1 $
    - $O$: 출력갚 높이/너비
    - $W$: 입력값 높이/너비
    - $K$: 필터 크기 (커널 크기)
    - $P$: 패딩
        - **동일 패딩 (non-zero)
        - $P = \frac{k-1}{2}$
    - $S$ : stride

#### Example 1: Output Dimension Calculation for Valid Padding

<img src = "./images/09-35.png">

- $W = 4$
- $K = 3$
- $P = 0$
- $S = 1$
- $O = \frac{4-3+2*0}{1} + 1 = \frac{1}{1} + 1 = 1 + 1 = 2$

#### Example 2: Output Dimension Calculation for Same Padding

<img src = "./images/09-27.png">

- $W = 5$
- $K = 3$
- $P = \frac{3-1}{2} = \frac{2}{2} = 1$
- $S = 1$
- $O = \frac{5-3+2*1}{1} + 1 = \frac{4}{1} + 1 = 5$

## 2. PyTorch를 이용하여 합성곱 네트워크 만들기

### 모델 A:

- 2 합성곱 계층
    - 동일 패딩 (동일한 출력 사이즈)
- 2 최대값 풀링 계층들
- 1 Fully Connected Layer

<img src = "./images/09-36.png">

### 단계
- Step 1: 데이터셋 로드
- Step 2: 데이터셋 순환 가능하게 만들기
- Step 3: 모델 클래스 생성
- Step 4: 모델 클래스 인스턴스화
- Step 5: 손실 클래스 인스턴스화
- Step 6: 최적화 클래스 인스턴스화
- Step 7: 모델 학습

### Step 1: MNIST 학습 Dataset 로드
#### 0에서 9까지의 이미지들

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
train_dataset = dsets.MNIST(root = './data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)

test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())

In [3]:
print (train_dataset.train_data.size())

torch.Size([60000, 28, 28])


In [4]:
print (train_dataset.train_labels.size())

torch.Size([60000])


In [5]:
print (test_dataset.test_data.size())

torch.Size([10000, 28, 28])


In [6]:
print (test_dataset.test_labels.size())

torch.Size([10000])


### Step 2: 데이터셋 순환 가능하게 만들기

In [7]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Step3: 클래스 만들기

<img src = "./images/09-36.png">

#### 합성곱을 위한 출력값 공식
- $O = \frac{W-K+2P}{S} + 1$
    - $O$: 출력갚 높이/너비
    - $W$: 입력값 높이/너비
    - $K$: **필터 사이즈 (커널 사이즈) = 5**
    - $P$: **동등 패딩 (non-zero)
        - $P = \frac{k-1}{2} = \frac{5-1}{2} = 2$ 
    - $S$: **stride = 1 **

#### 풀링을 위한 출력값 공식
- $O = \frac{W}{K}$
    - W: input height/width
    - K: **filter size = 2**

<img src = "./images/09-37.png">

In [8]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # 합성곱 1
        # in_channels=1 => MNIST 이미지는 흑백 이미지니까
        # out_channels = 16 개의 피처 맵들
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # 최대값 pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # 합성곱 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # 최대값 pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        # 32 * 7 * 7 is calculated above
        self.fc1 = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        # 합성곱 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # 최대값 pool 1
        out = self.maxpool1(out)
        
        # 합성곱 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # 최대값 pool 2
        out = self.maxpool2(out)
        
        # Resize
        # Original size : (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)
        
        # 선형 함수 (readout)
        out = self.fc1(out)
        
        return out        

### Step 4: 모델 클래스 인스턴스화

In [9]:
model = CNNModel()

### Step 5: 손실 클래스 인스턴스화

- 합성곱 뉴럴 네트워크: **Cross Entropy Loss**
    - *순방향 뉴럴 네트워크*: **Cross Entropy Loss**
    - *로지스틱 회귀분석*: **Cross Entropy Loss**
    - *선형 회귀분석*: **MSE**

In [10]:
criterion = nn.CrossEntropyLoss()

### Step 6: 최적화 클래스 인스턴스화

- 간단한 수식
    - $\theta = \theta - \eta \cdot \nabla_{\theta}$
        - $\theta$: 파라미터들 (우리의 변수들)
        - $\eta$: 학습률 (얼마나 빠르게 우리가 배울 수 있는가)
        - $\nabla_{\theta}$: 파라미터들의 그라디언트들
        
    - 더욱 간단화된 수식
        - 파라미터들 = 파라미터들 - 학습률 * 파라미터들의 그라디언트들
        - **매 iteration 마다, 우리는 모델의 파라미터들을 갱신한다**

In [11]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 파라미터들을 자세히 들여다보기

In [12]:
print (model.parameters())

<generator object Module.parameters at 0x10e53f570>


In [13]:
print (len(list(model.parameters())))

6


In [14]:
# 합성곱 1: 16 커널들
# 16 커널들이 첫번째 합성곱에 있고 크기는 5 * 5이다
# MNIST에서 입력값은 1 이다
print (list(model.parameters())[0].size())

torch.Size([16, 1, 5, 5])


In [15]:
# 합성곱 1 편차: 16 커널들
print (list(model.parameters())[1].size())

torch.Size([16])


In [16]:
# 합성곱 2: 깊이 = 16인 커널들 32개
# 커널 깊이는 입력값 크기에 따라 좌우된다(16 14 14)
print (list(model.parameters())[2].size())

torch.Size([32, 16, 5, 5])


In [17]:
# 합성곱 2 편차: 깊이=16인 커널들 32개
print (list(model.parameters())[3].size())

torch.Size([32])


In [18]:
# Fully Connected Layer 1
print (list(model.parameters())[4].size())

torch.Size([10, 1568])


In [19]:
# Fully Connected Layer 편차
print (list(model.parameters())[5].size())

torch.Size([10])


### Step 7: 모델 학습하기

- 과정
    1. **입력값/라벨들을 변수화**
        - CNN 입력값: (1, 28, 28)
        - Feedforward NN 입력값: (1, 28\*28)
    2. 그라디언트 버퍼들 비우기
    3. 주어진 입력값으로 출력값 구하기
    4. 손실 구하기
    5. 파라미터들에 관해 그라디언트들 구하기
    6. 그라디언트들을 이용하여 파라미터 업데이트하기
        - 파라미터들 = 파라미터들 - 학습률 * 파라미터들의 그라디언트들
    7. 반복

In [20]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 이미지들을 변수화
        # 차원을 변경할 이유가 없다, images.view(-1, 28*28)를 안 해도 된다
        # images.shape = (100, 1, 28, 28)
        images = Variable(images)
        labels = Variable(labels)
        
        # 파라미터들에 대한 그라디언트들 버퍼 비우기
        optimizer.zero_grad()
        
        # Forward 전달하여 출력값/logits값 얻기
        outputs = model(images)
        
        # 손실 계산: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # 파라미터들에 대한 그라디언트들 구하기
        loss.backward()
        
        # 파라미터들 업데이트
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # 정확도 계산
            correct = 0
            total = 0
            # 테스트 데이터셋 순환시키기
            for images, labels in test_loader:
                # 이미지들을 Torch Variable 형태로 로드
                images = Variable(images)
                
                # logits/출력값을 얻기 위해 Forward 패스
                outputs = model(images)
                
                # 가장 큰 값을 통해서 예측하기
                _, predicted = torch.max(outputs.data, 1)
                
                # 라벨들의 총 수
                total += labels.size(0)
                
                # 맞춘 예측의 수
                correct += (predicted == labels).sum()
            
            accuracy = 100 * int(correct) / int(total)
            
            # 로스값 구하기
            print (f'Iteration: {iter}, Loss: {loss.item()}, Accuracy: {accuracy}')      

Iteration: 500, Loss: 0.40629199147224426, Accuracy: 89.87
Iteration: 1000, Loss: 0.31727126240730286, Accuracy: 92.81
Iteration: 1500, Loss: 0.15736547112464905, Accuracy: 94.16
Iteration: 2000, Loss: 0.2962956130504608, Accuracy: 95.36
Iteration: 2500, Loss: 0.21549661457538605, Accuracy: 95.77
Iteration: 3000, Loss: 0.08201059699058533, Accuracy: 96.84


### 모델 B:

- 2층 합성곱
    - 동등 패딩 (같은 출력값 크기)
- 2층 **평균 풀링**
- 1 Fully Connected Layer

<img src="./images/09-38.png">

<img src="./images/09-39.png">

### 단계
- Step 1: 데이터셋 로드
- Step 2: 데이터셋 순환 가능하게 만들기
- Step 3: 모델 클래스 생성
- Step 4: 모델 클래스 인스턴스화
- Step 5: 손실 클래스 인스턴스화
- Step 6: 최적화 클래스 인스턴스화
- Step 7: 모델 학습

In [21]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

'''
STEP 3: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # 합성곱 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # 평균 풀링 1
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        
        # 합성곱 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # 평균 풀링 2
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        # 합성곱 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # 평균 풀링 1
        out = self.avgpool1(out)
        
        # 합성곱 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # 평균 풀링 2
        out = self.avgpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # 새로운 출력 크기: (100, 32*7*7)
        out = out.view(out.size(0), -1)
        
        # 선형 함수 (readout)
        out = self.fc1(out)
        
        return out
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        
        # Load images as Variable
        images = Variable(images)
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * int(correct) / int(total)
            
            # Print Loss
            print (f'Iteration: {iter}, Loss: {loss.item()}, Accuracy: {accuracy}')                   

Iteration: 500, Loss: 0.47322577238082886, Accuracy: 85.4
Iteration: 1000, Loss: 0.6276140809059143, Accuracy: 89.01
Iteration: 1500, Loss: 0.39957791566848755, Accuracy: 90.02
Iteration: 2000, Loss: 0.2400732785463333, Accuracy: 91.76
Iteration: 2500, Loss: 0.2980958819389343, Accuracy: 92.13
Iteration: 3000, Loss: 0.3201776444911957, Accuracy: 93.27


### 평균 풀링 테스트 정확도 < 최대값 풀링 테스트 정확도 (일반적)

### 모델 C:

- 2 층 신경망
    - **Valid 패딩** (output size 더 작음)
- 2 **최대값 Pooling** 계층
- 1 Fully Connected Layer

<img src="./images/09-40.png">

<img src="./images/09-41.png">

### 단계
- Step 1: 데이터셋 로드
- Step 2: 데이터셋 순환 가능하게 만들기
- Step 3: 모델 클래스 생성
- Step 4: 모델 클래스 인스턴스화
- Step 5: 손실 클래스 인스턴스화
- Step 6: 최적화 클래스 인스턴스화
- Step 7: 모델 학습

In [22]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # 합성곱 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # 최대값 pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # 합성곱 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # 최대값 pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        # 합성곱 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # 최대값 pool 1
        out = self.maxpool1(out)
        
        # 합성곱 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # 최대값 pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # 새로운 출력 크기: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        
        # Load images as Variable
        images = Variable(images)
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * int(correct) / int(total)
            
            # Print Loss
            print (f'Iteration: {iter}, Loss: {loss.item()}, Accuracy: {accuracy}')                   

Iteration: 500, Loss: 0.36808979511260986, Accuracy: 89.26
Iteration: 1000, Loss: 0.2121238261461258, Accuracy: 92.28
Iteration: 1500, Loss: 0.21809402108192444, Accuracy: 95.04
Iteration: 2000, Loss: 0.16695797443389893, Accuracy: 95.52
Iteration: 2500, Loss: 0.11453404277563095, Accuracy: 96.12
Iteration: 3000, Loss: 0.1668114811182022, Accuracy: 96.67


### Summary of Results

| Model A | Model B | Model C |
|------|------|------|
|Max Pooling|Average Pooling|Max Pooling|
|Same Padding|Same Padding|Valid Padding|
|96.72%|93.69%|96.61%|

| All Models |
|-----------|
|INPUT $\rightarrow$ CONV $\rightarrow$ POOL $\rightarrow$ CONV $\rightarrow$ POOL $\rightarrow$ FC|
|Convolution Kernel Size = 5 $\times$ 5|
|Convolution Kernel Stride = 1|
|Pooling Kernel Size = 2 $\times$ 2|

### Deep Learning
- 합성곱 신경망을 확장시키는 3가지 방법
    - 더 많은 합성곱계층
    - 보다 덜한 다운샘플링
        - 풀링을 위한 커널 사이즈를 줄인다 (점진적인 다운 셈플링)
    - 더 많은 fully connected layers
- 단점들
    - 더 큰 데이터셋이 요구됨
        - 차원의 저주
    - 보다 높은 정확도를 보장해 주지는 않음

## 3. PyTorch를 통해 CNN 구현하기 (GPU)

### 모델 A

<img src = "./images/09-36.png">

<img src = "./images/09-37.png">

GPU: GPU를 사용하기 위해 on이 되어야 하는 2가지
- model
- variables

### 단계
- Step 1: 데이터셋 로드
- Step 2: 데이터셋 순환 가능하게 만들기
- Step 3: 모델 클래스 생성
- **Step 4: 모델 클래스 인스턴스화**
- Step 5: 손실 클래스 인스턴스화
- Step 6: 최적화 클래스 인스턴스화
- **Step 7: 모델 학습**

In [23]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''

model = CNNModel()

#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    model.cuda()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.cuda())
                else:
                    images = Variable(images)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * int(correct) / int(total)
            
            # Print Loss
            print (f'Iteration: {iter}, Loss: {loss.item()}, Accuracy: {accuracy}')

Iteration: 500, Loss: 0.5421700477600098, Accuracy: 87.83
Iteration: 1000, Loss: 0.3613150417804718, Accuracy: 92.35
Iteration: 1500, Loss: 0.1518508940935135, Accuracy: 94.69
Iteration: 2000, Loss: 0.19660723209381104, Accuracy: 95.48
Iteration: 2500, Loss: 0.2636498510837555, Accuracy: 96.36
Iteration: 3000, Loss: 0.1706184446811676, Accuracy: 96.81


### 요약

- **Feedforward Neural Network** 에서의 변환
    - 선형 계층들 이전에 ** 합성곱 & 풀링** 계층들 추가
- 하나의 **합성곱** 계층 기초들
- 하나의 **풀링** 계층 기초들
    - 최대값 pooling
    - 평균값 pooling
- **패딩**
- **출력 차원** 계산과 예들
    - $O = \frac{W-K+2P}{S}+1$
- 합성곱 뉴럴 네트워크들
    - **모델 A** : 2 Conv + 2 Max pool + 1 FC
        - Same Padding
    - **모델 B** : 2 Conv + 2 Average pool + 1 FC
        - Same Padding
    - **모델 C** : C Conv + 2 Max pool + 1 FC
        - Valid Padding
    - **코드**에서 모델의 변형
        - Modifying only step 3
    - 모델의 **수용능력**을 확장시킬 수 있는 방법들
        - 더 많은 합성곱들
        - 점진적인 pooling
        - 더 많은 Fully connected 계층들
    - **GPU** Code
        - 2 things on GPU
            - model
            - variable
        - **Step 4 & Step 7**만 수정하면 됨
    - **7 단계 ** 모델 구현 요약
        - Step 1: 데이터셋 로드
        - Step 2: 데이터셋 순환 가능하게 만들기
        - Step 3: 모델 클래스 만들기
        - Step 4: 모델 클래스 인스턴스화 하기
        - Step 5: 손실 클래스 인스턴스화 하기
        - Step 6: 최적화 클래스 인스턴스화 하기
        - Step 7: 모델 학습하기